<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllCF_Add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('all')
from nltk.corpus import sentiwordnet as swn
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower_clean.csv')
df = df.sample(frac=1,random_state=32)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [ ]:
df.head()

,Unnamed: 0,emotion,tweets
35975,35975,6,katofawesome praying for love in a lap dance a...
17590,17590,1,trying to fix my internet connectionguess my p...
38071,38071,0,starting an account here on twitter
23304,23304,1,dipfico hmm wrong link ignore my tweet
26990,26990,0,mzunyque thanks before the major chop


In [ ]:
df.emotion.unique()

array([ 6,  1,  0,  2,  3,  4, 10,  5,  7,  8,  9, 12, 11])

In [ ]:
category_dict = {0:[1,0,0,0,0,0,0,0,0,0,0,0,0],1:[0,1,0,0,0,0,0,0,0,0,0,0,0],2:[0,0,1,0,0,0,0,0,0,0,0,0,0],3:[0,0,0,1,0,0,0,0,0,0,0,0,0],4:[0,0,0,0,1,0,0,0,0,0,0,0,0],5:[0,0,0,0,0,1,0,0,0,0,0,0,0],6:[0,0,0,0,0,0,1,0,0,0,0,0,0],7:[0,0,0,0,0,0,0,1,0,0,0,0,0],8:[0,0,0,0,0,0,0,0,1,0,0,0,0],9:[0,0,0,0,0,0,0,0,0,1,0,0,0],10:[0,0,0,0,0,0,0,0,0,0,1,0,0],11:[0,0,0,0,0,0,0,0,0,0,0,1,0],12:[0,0,0,0,0,0,0,0,0,0,0,0,1]}
data_tweet = [x.lower().split() for x in df['tweets']]
data_cat = np.array([category_dict[x] for x in df['emotion']])

In [ ]:
print(data_tweet[:5])
print(data_cat[:5])

[['katofawesome', 'praying', 'for', 'love', 'in', 'a', 'lap', 'dance', 'and', 'paying', 'in', 'naivety', 'lt--one', 'of', 'my', 'fav', 'songs'], ['trying', 'to', 'fix', 'my', 'internet', 'connectionguess', 'my', 'prayers', 'have', 'been', 'answered', 'and', 'i', 'wont', 'have', 'any', 'study', 'distractions', 'ugh'], ['starting', 'an', 'account', 'here', 'on', 'twitter'], ['dipfico', 'hmm', 'wrong', 'link', 'ignore', 'my', 'tweet'], ['mzunyque', 'thanks', 'before', 'the', 'major', 'chop']]
[[0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]
def getsent(word,tag):
  res=0
  try:
    x = swn.senti_synset(word+'.'+tag[0].lower()+'.01')
    res =  (x.pos_score()-x.neg_score())
  finally:
    return res 
data_tweet = [[(i[0],i[1],getsent(i[0],i[1])) for i in x] for x in data_tweet]

In [ ]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-1,0),min(i+2,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])/2,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
result_table = [0,0,0,0,0,0,0,0,0]

In [ ]:
epochs = 40

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_lstm = model_sg.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 92s 352ms/step - loss: 2.2628 - accuracy: 0.2098 - val_loss: 2.1376 - val_accuracy: 0.2182
Epoch 2/40
257/257 [==============================] - 89s 346ms/step - loss: 2.1417 - accuracy: 0.2298 - val_loss: 2.0951 - val_accuracy: 0.2617
Epoch 3/40
257/257 [==============================] - 89s 346ms/step - loss: 2.0946 - accuracy: 0.2605 - val_loss: 2.0719 - val_accuracy: 0.2752
Epoch 4/40
257/257 [==============================] - 89s 348ms/step - loss: 2.0579 - accuracy: 0.2804 - val_loss: 2.0557 - val_accuracy: 0.2883
Epoch 5/40
257/257 [==============================] - 90s 350ms/step - loss: 2.0331 - accuracy: 0.2968 - val_loss: 2.0436 - val_accuracy: 0.2923
Epoch 6/40
257/257 [==============================] - 90s 351ms/step - loss: 2.0218 - accuracy: 0.3017 - val_loss: 2.0307 - val_accuracy: 0.2922
Epoch 7/40
257/257 [==============================] - 94s 367ms/step - loss: 2.0071 - accuracy: 0.3067 - val_loss: 2.0273 - val_ac

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 107s 403ms/step - loss: 2.2408 - accuracy: 0.2211 - val_loss: 2.0670 - val_accuracy: 0.2771
Epoch 2/40
257/257 [==============================] - 99s 387ms/step - loss: 2.0593 - accuracy: 0.2859 - val_loss: 2.0313 - val_accuracy: 0.2986
Epoch 3/40
257/257 [==============================] - 100s 391ms/step - loss: 2.0220 - accuracy: 0.3057 - val_loss: 2.0193 - val_accuracy: 0.3083
Epoch 4/40
257/257 [==============================] - 100s 391ms/step - loss: 2.0029 - accuracy: 0.3158 - val_loss: 2.0140 - val_accuracy: 0.3109
Epoch 5/40
257/257 [==============================] - 100s 389ms/step - loss: 1.9860 - accuracy: 0.3211 - val_loss: 2.0112 - val_accuracy: 0.3125
Epoch 6/40
257/257 [==============================] - 101s 392ms/step - loss: 1.9705 - accuracy: 0.3264 - val_loss: 2.0097 - val_accuracy: 0.3126
Epoch 7/40
257/257 [==============================] - 100s 392ms/step - loss: 1.9556 - accuracy: 0.3328 - val_loss: 2.0077 - 

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_rnn = model_sg_rnn.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)


result_table[2]=history[1]
print("SG_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 87s 334ms/step - loss: 2.2733 - accuracy: 0.1824 - val_loss: 2.1413 - val_accuracy: 0.2157
Epoch 2/40
257/257 [==============================] - 84s 326ms/step - loss: 2.1427 - accuracy: 0.2170 - val_loss: 2.1368 - val_accuracy: 0.2154
Epoch 3/40
257/257 [==============================] - 83s 324ms/step - loss: 2.1437 - accuracy: 0.2153 - val_loss: 2.1378 - val_accuracy: 0.2158
Epoch 4/40
257/257 [==============================] - 82s 321ms/step - loss: 2.1441 - accuracy: 0.2158 - val_loss: 2.1396 - val_accuracy: 0.2154
Epoch 5/40
257/257 [==============================] - 84s 328ms/step - loss: 2.1413 - accuracy: 0.2144 - val_loss: 2.1386 - val_accuracy: 0.2158
Epoch 6/40
257/257 [==============================] - 83s 325ms/step - loss: 2.1418 - accuracy: 0.2165 - val_loss: 2.1387 - val_accuracy: 0.2132
Epoch 7/40
257/257 [==============================] - 83s 324ms/step - loss: 2.1402 - accuracy: 0.2218 - val_loss: 2.1396 - val_ac

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_lstm = model_cbow.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 91s 350ms/step - loss: 2.2548 - accuracy: 0.2135 - val_loss: 2.1324 - val_accuracy: 0.2212
Epoch 2/40
257/257 [==============================] - 88s 345ms/step - loss: 2.1360 - accuracy: 0.2216 - val_loss: 2.0945 - val_accuracy: 0.2509
Epoch 3/40
257/257 [==============================] - 90s 350ms/step - loss: 2.0946 - accuracy: 0.2455 - val_loss: 2.0739 - val_accuracy: 0.2628
Epoch 4/40
257/257 [==============================] - 91s 354ms/step - loss: 2.0614 - accuracy: 0.2748 - val_loss: 2.0580 - val_accuracy: 0.2835
Epoch 5/40
257/257 [==============================] - 91s 354ms/step - loss: 2.0381 - accuracy: 0.2941 - val_loss: 2.0448 - val_accuracy: 0.2951
Epoch 6/40
257/257 [==============================] - 92s 357ms/step - loss: 2.0252 - accuracy: 0.3054 - val_loss: 2.0324 - val_accuracy: 0.2985
Epoch 7/40
257/257 [==============================] - 90s 353ms/step - loss: 2.0100 - accuracy: 0.3135 - val_loss: 2.0289 - val_ac

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 108s 406ms/step - loss: 2.2421 - accuracy: 0.2274 - val_loss: 2.0779 - val_accuracy: 0.2748
Epoch 2/40
257/257 [==============================] - 99s 385ms/step - loss: 2.0628 - accuracy: 0.2841 - val_loss: 2.0411 - val_accuracy: 0.2949
Epoch 3/40
257/257 [==============================] - 114s 445ms/step - loss: 2.0274 - accuracy: 0.2988 - val_loss: 2.0286 - val_accuracy: 0.3012
Epoch 4/40
257/257 [==============================] - 116s 454ms/step - loss: 2.0105 - accuracy: 0.3090 - val_loss: 2.0253 - val_accuracy: 0.3015
Epoch 5/40
257/257 [==============================] - 121s 474ms/step - loss: 1.9928 - accuracy: 0.3167 - val_loss: 2.0250 - val_accuracy: 0.3040
Epoch 6/40
257/257 [==============================] - 114s 445ms/step - loss: 1.9762 - accuracy: 0.3220 - val_loss: 2.0243 - val_accuracy: 0.3040
Epoch 7/40
257/257 [==============================] - 119s 465ms/step - loss: 1.9599 - accuracy: 0.3296 - val_loss: 2.0280 - 

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_rnn = model_cbow_rnn.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 85s 330ms/step - loss: 2.2501 - accuracy: 0.2144 - val_loss: 2.1239 - val_accuracy: 0.2360
Epoch 2/40
257/257 [==============================] - 84s 327ms/step - loss: 2.1019 - accuracy: 0.2549 - val_loss: 2.1109 - val_accuracy: 0.2331
Epoch 3/40
257/257 [==============================] - 85s 331ms/step - loss: 2.0866 - accuracy: 0.2632 - val_loss: 2.0902 - val_accuracy: 0.2475
Epoch 4/40
257/257 [==============================] - 83s 325ms/step - loss: 2.0689 - accuracy: 0.2757 - val_loss: 2.0704 - val_accuracy: 0.2765
Epoch 5/40
257/257 [==============================] - 84s 327ms/step - loss: 2.0510 - accuracy: 0.2832 - val_loss: 2.0759 - val_accuracy: 0.2785
Epoch 6/40
257/257 [==============================] - 84s 327ms/step - loss: 2.0396 - accuracy: 0.2891 - val_loss: 2.0859 - val_accuracy: 0.2735
Epoch 7/40
257/257 [==============================] - 84s 328ms/step - loss: 2.0276 - accuracy: 0.2957 - val_loss: 2.0868 - val_ac

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_lstm = model_glove.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 17s 59ms/step - loss: 2.2303 - accuracy: 0.2244 - val_loss: 1.9843 - val_accuracy: 0.3057
Epoch 2/40
257/257 [==============================] - 14s 56ms/step - loss: 1.9681 - accuracy: 0.3176 - val_loss: 1.9079 - val_accuracy: 0.3377
Epoch 3/40
257/257 [==============================] - 14s 56ms/step - loss: 1.9109 - accuracy: 0.3376 - val_loss: 1.8908 - val_accuracy: 0.3443
Epoch 4/40
257/257 [==============================] - 14s 55ms/step - loss: 1.8779 - accuracy: 0.3578 - val_loss: 1.8796 - val_accuracy: 0.3471
Epoch 5/40
257/257 [==============================] - 14s 54ms/step - loss: 1.8583 - accuracy: 0.3676 - val_loss: 1.8787 - val_accuracy: 0.3463
Epoch 6/40
257/257 [==============================] - 14s 54ms/step - loss: 1.8444 - accuracy: 0.3729 - val_loss: 1.8718 - val_accuracy: 0.3535
Epoch 7/40
257/257 [==============================] - 14s 56ms/step - loss: 1.8235 - accuracy: 0.3816 - val_loss: 1.8730 - val_accuracy:

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_rnn = model_glove_rnn.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/40


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


257/257 [==============================] - 7s 26ms/step - loss: 2.2235 - accuracy: 0.1948 - val_loss: 2.1395 - val_accuracy: 0.2146
Epoch 2/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1516 - accuracy: 0.2178 - val_loss: 2.1370 - val_accuracy: 0.2191
Epoch 3/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1474 - accuracy: 0.2301 - val_loss: 2.1259 - val_accuracy: 0.2462
Epoch 4/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1404 - accuracy: 0.2401 - val_loss: 2.1419 - val_accuracy: 0.2171
Epoch 5/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1566 - accuracy: 0.2172 - val_loss: 2.1385 - val_accuracy: 0.2175
Epoch 6/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1540 - accuracy: 0.2157 - val_loss: 2.1402 - val_accuracy: 0.2174
Epoch 7/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1540 - accuracy: 0.2170 - val_loss: 2.1392 - val_accuracy: 0.2175
Epoch 8/40

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 28s 97ms/step - loss: 2.1339 - accuracy: 0.2789 - val_loss: 1.9030 - val_accuracy: 0.3442
Epoch 2/40
257/257 [==============================] - 24s 94ms/step - loss: 1.8888 - accuracy: 0.3522 - val_loss: 1.8603 - val_accuracy: 0.3555
Epoch 3/40
257/257 [==============================] - 24s 94ms/step - loss: 1.8410 - accuracy: 0.3729 - val_loss: 1.8443 - val_accuracy: 0.3605
Epoch 4/40
257/257 [==============================] - 24s 93ms/step - loss: 1.8092 - accuracy: 0.3802 - val_loss: 1.8384 - val_accuracy: 0.3675
Epoch 5/40
257/257 [==============================] - 24s 92ms/step - loss: 1.7730 - accuracy: 0.3918 - val_loss: 1.8382 - val_accuracy: 0.3695
Epoch 6/40
257/257 [==============================] - 24s 93ms/step - loss: 1.7480 - accuracy: 0.4048 - val_loss: 1.8430 - val_accuracy: 0.3706
Epoch 7/40
257/257 [==============================] - 24s 94ms/step - loss: 1.7174 - accuracy: 0.4170 - val_loss: 1.8563 - val_accuracy:

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

	Lstm			BiLstm			Rnn
sg [0.3062500059604645, 0.2771250009536743, 0.23925000429153442]
cbow [0.30524998903274536, 0.28349998593330383, 0.23549999296665192]
glove [0.3086250126361847, 0.28337499499320984, 0.20524999499320984]
